In [1]:
import pandas as pd
import os
import json
from itertools import chain
from collections import defaultdict
import dask.dataframe as dd
import time

In [2]:
new_yarn = pd.read_csv('new_yarn.csv')
synsets_with_no_definitions = pd.read_csv('synsets_with_no_defs')

In [3]:
new_yarn = pd.concat([new_yarn, synsets_with_no_definitions], sort=False)

In [4]:
del synsets_with_no_definitions

In [60]:
new_yarn.reset_index(inplace=True)

In [5]:
new_yarn.iloc[0]

Unnamed: 0                                     0
triple_merged_by                      1797;39963
words                автомобиль;драндулет;машина
def_ids                         1797;71159;39963
length_in_words                                3
length_in_def_ids                              3
yarn_id                                      NaN
Name: 0, dtype: object

In [6]:
new_yarn.shape

(99505, 7)

In [7]:
new_yarn = dd.from_pandas(new_yarn, npartitions=4)

In [8]:
def find_dict_synsets(dict_synsets_by_length, yarn_synset):
    yarn_synset = set(yarn_synset.split(';'))
    return [';'.join(s) for i in range(1, len(yarn_synset)+1) for s in dict_synsets_by_length[i] if s.issubset(yarn_synset)]

In [9]:
for dictionary in ['ruTes', 'babenko', 'evgenieva']:
    print('Processing {}'.format(dictionary))
    raw = []
    with open(os.path.join('..', 'db', 'data', 'dicts', '{}_final.json'.format(dictionary)), encoding='utf8') as f:
        for line in f:
            raw.append(json.loads(line))
    print('Raw lines: {}'.format(len(raw)))
    raw_synsets = set()
    for entry in raw:
        if 'synonym' in entry:
            for syn in entry['synonym']:
                raw_synsets.add(';'.join(sorted(x.lower() for x in chain(syn, entry['word']))))
    print('Unique synsets: {}'.format(len(raw_synsets)))
    synsets_by_length = defaultdict(list)
    for s in raw_synsets:
        synset = set(s.split(';'))
        synsets_by_length[len(synset)].append(synset)
        
    new_yarn[dictionary] = new_yarn.map_partitions(lambda df: df.apply(lambda x: find_dict_synsets(synsets_by_length, 
                                                                                                                x.words), 
                                                                 axis=1)).compute(scheduler='processes')
    print()

Processing ruTes
Raw lines: 96700
Unique synsets: 22722

Processing babenko
Raw lines: 4988
Unique synsets: 4864

Processing evgenieva
Raw lines: 5441
Unique synsets: 5429



In [11]:
new_yarn = new_yarn.compute()

In [82]:
new_yarn.head(10)

,index,Unnamed: 0,triple_merged_by,words,def_ids,length_in_words,length_in_def_ids,yarn_id,ruTes,babenko,evgenieva,ruTes_diff,babenko_diff,evgenieva_diff
0,0,0,1797;39963,автомобиль;драндулет;машина,1797;71159;39963,3.0,3.0,NaN,[],[],[],NaN,NaN,NaN
1,0,8,NaN,колёса,NaN,NaN,NaN,1.0,[],[],[],NaN,NaN,NaN
2,1,2,39963;71159,автомобиль;драндулет;машина;прибор,1797;71159;39963;108163,4.0,4.0,NaN,[],[],[],NaN,NaN,NaN
3,1,104,NaN,дружбан,NaN,NaN,NaN,13.0,[],[],[],NaN,NaN,NaN
4,2,3,1549;1774,авто;автомашина;автомобиль;кабриолет;легковая ...,148085;1774;66305;1549;71160;1797;54389;167404...,10.0,9.0,NaN,[],[],[автомобиль;машина;авто;автомашина],NaN,NaN,6.0
5,2,132,NaN,в преклонном возрасте;в почтенных летах,NaN,NaN,NaN,17.0,[],[],[],NaN,NaN,NaN
6,3,6,71161,машина,71161,1.0,1.0,NaN,[],[],[],NaN,NaN,NaN
7,3,1058,NaN,материнка,NaN,NaN,NaN,112.0,[],[],[],NaN,NaN,NaN
8,4,7,179054,тачка,179054,1.0,1.0,NaN,[],[],[],NaN,NaN,NaN
9,4,1320,NaN,начальная школа,NaN,NaN,NaN,149.0,[],[],[],NaN,NaN,NaN


## ruTes

In [24]:
new_yarn[new_yarn.ruTes.map(len) >= 2][['words', 'ruTes']]

,words,ruTes
97,авторитет;благородство;важность;величавость;до...,"[значение;значимость, пальма первенства;первен..."
99,важность;значение;значимость;значительность;ро...,"[значение;значимость, значение;ценность;роль;в..."
953,капелька;капля;кроха;крохотка;крошка;крупинка;...,"[кроха;крошка, крупинка;крупица, крошка;крупиц..."
1056,дорога;манера;норма;образ;обряд;организация;оч...,"[упорядоченность;порядок, образ;средство;способ]"
1239,звание;категория;класс;разряд;ранг;степень;точ...,"[звание;ранг, класс;ранг]"
1242,звание;категория;класс;разряд;ранг;степень;уро...,"[звание;ранг, класс;ранг]"
1244,вид;гильдия;группа;жанр;звание;каста;категория...,"[звание;ранг, класс;ранг]"
1926,артист;дока;знаток;мастер;специалист;эксперт,"[специалист;эксперт, дока;знаток]"
3033,артист;дока;знаток;знающий;искусный;мастер;спе...,"[специалист;эксперт, дока;знаток]"
3037,дока;знаток;мастер;специалист;эксперт,"[специалист;эксперт, дока;знаток]"


In [40]:
def yarn_dict_diff(row, dict_name):
    if len(row[dict_name]) == 0:
        return None
    return len(row.words.split(';')) - len(row[dict_name][0].split(';'))

new_yarn['ruTes_diff'] = new_yarn.apply(lambda row: yarn_dict_diff(row, 'ruTes'), axis=1)
new_yarn['babenko_diff'] = new_yarn.apply(lambda row: yarn_dict_diff(row, 'babenko'), axis=1)
new_yarn['evgenieva_diff'] = new_yarn.apply(lambda row: yarn_dict_diff(row, 'evgenieva'), axis=1)

In [47]:
new_yarn[new_yarn.ruTes_diff.astype(bool) & (new_yarn.ruTes_diff > 0)].ruTes_diff.describe()

count    388.000000
mean       7.030928
std        6.508517
min        1.000000
25%        2.000000
50%        5.000000
75%       10.000000
max       33.000000
Name: ruTes_diff, dtype: float64

In [87]:
new_yarn[(new_yarn.ruTes.map(len) != 0) & (new_yarn.ruTes_diff > 0) & (new_yarn.words.str.split(';').str.len() <= 3)].ruTes_diff.describe()

count    62.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
Name: ruTes_diff, dtype: float64

In [61]:
new_yarn[new_yarn.ruTes_diff == 7][['words', 'ruTes']].head()

,words,ruTes
1249,век;время;период;полоса;промежуток времени;сро...,[эра;век;эпоха]
5997,бенефис;действие;действо;зрелище;комедия;лицед...,[действо;представление;постановка]
5998,действие;действо;зрелище;картина;комедия;лицед...,[картина;зрелище]
6065,артист;виртуоз;дока;мастер;профессионал;профес...,[специалист;эксперт]
6105,аппаратура;арматура;инвентарь;инструментарий;о...,[инвентарь;оборудование]


In [62]:
new_yarn.loc[6105].words, new_yarn.loc[6105].ruTes

('аппаратура;арматура;инвентарь;инструментарий;оборудование;оснастка;оснащение;снаряжение;снасть',
 ['инвентарь;оборудование'])

In [45]:
new_yarn[new_yarn.ruTes.astype(bool)].shape

(531, 13)

## babenko

In [66]:
print('Для {} синсетов подобраны синсеты Бабенко'.format(new_yarn[new_yarn.babenko.astype(bool)].shape[0]))

Для 1249 синсетов подобраны синсеты Бабенко


In [78]:
new_yarn[new_yarn.babenko.map(len) >= 2].babenko_diff.describe()

count    90.000000
mean      9.066667
std       8.535188
min       1.000000
25%       4.000000
50%       6.500000
75%      10.000000
max      37.000000
Name: babenko_diff, dtype: float64

In [71]:
new_yarn[(new_yarn.babenko.map(len) == 1) & (new_yarn.babenko_diff > 0)].ruTes_diff.describe()

count    108.000000
mean      10.731481
std        7.556153
min        1.000000
25%        5.000000
50%        9.000000
75%       15.250000
max       30.000000
Name: ruTes_diff, dtype: float64

In [86]:
new_yarn[(new_yarn.babenko.map(len) != 0) & (new_yarn.babenko_diff > 0) & (new_yarn.words.str.split(';').str.len() <= 3)].babenko_diff.describe()

count    127.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: babenko_diff, dtype: float64

In [73]:
new_yarn[new_yarn.babenko.astype(bool)].head(10)[['words', 'babenko']]

,words,babenko
131,грамота;письмо;послание,[письмо;послание]
132,грамота;грамотка;записка;письмо;послание,[письмо;послание]
159,выручка;опора;поддержка;подмога;подспорье;помо...,"[помощь;поддержка, помощь;содействие;поддержка]"
164,поддержка;помощь;пособие;услуга,[помощь;поддержка]
194,авторитет;благородство;важность;величавость;до...,[смысл;значение]
266,брюки;брючки;панталоны;портки;штаны,[штаны;брюки;панталоны]
404,гастроном;киоск;лабаз;магазин;супермаркет;унив...,[супермаркет;магазин;универмаг;универсам]
465,декрет;закон;постановление;приказ;приказание;р...,[распоряжение;приказ]
487,династия;дом;клан;порода;род;семейство;семья;ф...,[фамилия;дом;семья;семейство]
489,династия;дом;клан;род;семейство;семья;фамилия,[фамилия;дом;семья;семейство]


## Evgenieva

In [74]:
print('Для {} синсетов подобраны синсеты Евгеньевой'.format(new_yarn[new_yarn.evgenieva.astype(bool)].shape[0]))

Для 2133 синсетов подобраны синсеты Евгеньевой


In [77]:
new_yarn[new_yarn.evgenieva.map(len) >= 2].evgenieva_diff.describe()

count    231.000000
mean       7.753247
std        5.361268
min        1.000000
25%        4.000000
50%        6.000000
75%       11.000000
max       39.000000
Name: evgenieva_diff, dtype: float64

In [79]:
new_yarn[(new_yarn.evgenieva.map(len) == 1) & (new_yarn.evgenieva_diff > 0)].evgenieva_diff.describe()

count    1456.000000
mean        4.561126
std         4.203791
min         1.000000
25%         2.000000
50%         3.000000
75%         6.000000
max        36.000000
Name: evgenieva_diff, dtype: float64

In [85]:
new_yarn[(new_yarn.evgenieva.map(len) != 0) & (new_yarn.evgenieva_diff > 0) & (new_yarn.words.str.split(';').str.len() <= 3)].evgenieva_diff.describe()

count    265.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: evgenieva_diff, dtype: float64

In [80]:
new_yarn[new_yarn.evgenieva.astype(bool)].head(10)[['words', 'evgenieva']]

,words,evgenieva
4,авто;автомашина;автомобиль;кабриолет;легковая ...,[автомобиль;машина;авто;автомашина]
177,жизнь;конъюнктура;обстановка;обстоятельства;об...,[обстановка;обстоятельства;ситуация;условия]
194,авторитет;благородство;важность;величавость;до...,[смысл;значение]
240,выражение;высказывание;предложение;суждение;фраза,[предложение;суждение]
388,пугало;чучело,[чучело;пугало]
441,довод;обоснование;оправдание;основание;отговор...,"[смысл;резон, причина;резон;основание]"
443,мотив;оправдание;основание;отговорка;первоприч...,[причина;резон;основание]
445,мотив;оправдание;основание;первопричина;повод;...,[причина;резон;основание]
446,мотив;обоснование;оправдание;основание;отговор...,"[смысл;резон, причина;резон;основание]"
448,мотив;основание;первопричина;повод;причина;резон,[причина;резон;основание]


In [81]:
new_yarn.loc[194].words, new_yarn.loc[194].evgenieva

('авторитет;благородство;важность;величавость;добротность;достоинство;значение;значимость;пальма первенства;первенство;превосходство;предпочтение;преимущество;роль;сила;смысл;стоимость;ценность',
 ['смысл;значение'])